In [1]:
import numpy as np
import pandas as pd
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz
from astropy import units as u
from astropy import constants as const
from astropy.units import imperial
imperial.enable()

from astroplan import Observer, FixedTarget
%matplotlib inline
import matplotlib.pyplot as plt

from astroplan import time_grid_from_range

from astroplan.plots import plot_airmass
from astropy.table import Table


In [2]:
my_column_names = ["Name", "RAh", "RAm", "RAs", "DEd","DEm","DEs","FName","Type"]
df = pd.read_csv("./Navigation2/galaxytable.txt", skiprows = 24, delimiter= " ", header = None, names=my_column_names)
df.head()
df_random = df.sample(n=80)
df_random.head()

,Name,RAh,RAm,RAs,DEd,DEm,DEs,FName,Type
7515,7516,13,6,35.8,-5,10,47.7,RCS1303,M
8990,8991,15,31,24.5,6,27,44.0,RCS1514,M
4048,4049,3,5,6.3,-16,54,35.5,RCS0310,M
7011,7012,12,46,30.6,2,57,37.5,RCS1303,CM
7852,7853,15,2,26.9,9,4,24.8,RCS1514,M


In [3]:
#convert to same unit and add
dfr = Table.from_pandas(df_random)
RAh_h = dfr['RAh']*u.hourangle
RAm_h = (dfr['RAm'] / 60) * u.hourangle
RAs_h = (dfr['RAs'] / 3600) * u.hourangle

#add
RA = RAh_h + RAm_h + RAs_h

DEh_h = dfr['DEd']*u.deg
DEm_h = (dfr['DEm'] * u.arcmin).to(u.deg)
DEs_h = (dfr['DEs'] * u.arcsec).to(u.deg)

#add
DE = DEh_h + DEm_h + DEs_h
dfr

Name,RAh,RAm,RAs,DEd,DEm,DEs,FName,Type
int64,int64,int64,float64,int64,int64,float64,str7,str2
7516,13,6,35.8,-5,10,47.7,RCS1303,M
8991,15,31,24.5,6,27,44.0,RCS1514,M
4049,3,5,6.3,-16,54,35.5,RCS0310,M
7012,12,46,30.6,2,57,37.5,RCS1303,CM
7853,15,2,26.9,9,4,24.8,RCS1514,M
10710,21,25,37.5,0,42,47.7,RCS2143,M
14262,23,35,25.6,-9,57,15.5,RCS2338,M
8015,15,6,2.4,10,12,44.0,RCS1514,M
10456,16,57,51.1,37,4,20.4,RCS1645,M


In [4]:
dfr.remove_columns(['RAh','RAm', 'RAs', 'DEd','DEm','DEs','FName','Type'])
dfr['RA'] = RA
dfr['DE'] = DE
dfr

Name,RA,DE
,hourangle,deg
int64,float64,float64
7516,13.109944444444444,-4.820083333333333
8991,15.523472222222223,6.4622222222222225
4049,3.0850833333333334,-15.09013888888889
7012,12.775166666666667,2.9604166666666667
7853,15.040805555555556,9.073555555555556
10710,21.427083333333336,0.7132499999999999
14262,23.590444444444444,-8.045694444444445
8015,15.100666666666667,10.212222222222222


In [5]:
cfht = Observer(latitude = 19.8253 * u.deg,
               longitude = -155.47 * u.deg,
               elevation = 4204 * u.m,
               timezone = 'US/Hawaii',
               name = "CFHT"
              )
sep1 = Time("2019-09-01 12:00:00")

In [6]:
astro_set1 = cfht.twilight_evening_astronomical(sep1, which="nearest")
astro_rise1 = cfht.twilight_morning_astronomical(sep1, which="next")
observing_time = (astro_rise1 - astro_set1).to(u.h)

In [7]:
print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set1))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise1))
print("You can observe for {0:.1f} during the window".format(observing_time))




Astronomical Evening Twilight starts at 2019-09-01 05:52:38.177 UTC
Astronomical Morning Twilight starts at 2019-09-01 14:51:35.079 UTC
You can observe for 9.0 h during the window


In [8]:
coords = SkyCoord(dfr['RA'][0]*u.deg, dfr['DE'][0]*u.deg, frame='icrs')


In [9]:
#all observing window
sep2 = Time("2019-09-02 12:00:00")
sep3 = Time("2019-09-03 12:00:00")
sep4 = Time("2019-09-04 12:00:00")
sep5 = Time("2019-09-05 12:00:00")
sep6 = Time("2019-09-06 12:00:00")
sep7 = Time("2019-09-07 12:00:00")
sep8 = Time("2019-09-08 12:00:00")
sep9 = Time("2019-09-09 12:00:00")
sep10 = Time("2019-09-10 12:00:00")
sep11 = Time("2019-09-11 12:00:00")
sep12 = Time("2019-09-12 12:00:00")
sep13 = Time("2019-09-13 12:00:00")
sep14 = Time("2019-09-14 12:00:00")
sep15 = Time("2019-09-15 12:00:00")

times=[sep1,sep2,sep3,sep4,sep5,sep6,sep7,sep8,sep9,sep10,sep11,sep12,sep13,sep14,sep15]


In [18]:
#Airmass
altaz_my_target = []
coord_l = []
for x in range(len(dfr)):
    coord = SkyCoord(dfr['RA'][x]*u.deg, dfr['DE'][x]*u.deg, frame='icrs')
    coord_l.append(coord)
    altaz_my_target = cfht.altaz(astro_rise1,coord_l[x])
    dfr['airmass'] = altaz_my_target.secz
mask_l = dfr['airmass'] < 2.0
new = dfr[mask_l]

Name,RA,DE,airmass
,hourangle,deg,
int64,float64,float64,float64
7516,13.109944444444444,-4.820083333333333,1.8317278241187538
8991,15.523472222222223,6.4622222222222225,1.8317278241187538
4049,3.0850833333333334,-15.09013888888889,1.8317278241187538
7012,12.775166666666667,2.9604166666666667,1.8317278241187538
7853,15.040805555555556,9.073555555555556,1.8317278241187538
10710,21.427083333333336,0.7132499999999999,1.8317278241187538
14262,23.590444444444444,-8.045694444444445,1.8317278241187538
8015,15.100666666666667,10.212222222222222,1.8317278241187538


In [19]:
coord_list = []
mask = []
for x in range(len(new)):
    coords = SkyCoord(new['RA'][x]*u.deg, new['DE'][x]*u.deg, frame='icrs')
    coord_list.append(coords)
    
    mask.append(cfht.target_is_up(astro_rise1, coord_list[x]))
    
    
    

i = new[mask]

for j in range(len(i)):
    print("Object {0} with RA {1:.1f} hour and DEC {2:.1f} deg is visible during the observing window"
          .format(i[j][0],i[j][1],i[j][2]))

Object 7516 with RA 13.1 hour and DEC -4.8 deg is visible during the observing window
Object 8991 with RA 15.5 hour and DEC 6.5 deg is visible during the observing window
Object 4049 with RA 3.1 hour and DEC -15.1 deg is visible during the observing window
Object 7012 with RA 12.8 hour and DEC 3.0 deg is visible during the observing window
Object 7853 with RA 15.0 hour and DEC 9.1 deg is visible during the observing window
Object 10710 with RA 21.4 hour and DEC 0.7 deg is visible during the observing window
Object 14262 with RA 23.6 hour and DEC -8.0 deg is visible during the observing window
Object 8015 with RA 15.1 hour and DEC 10.2 deg is visible during the observing window
Object 10456 with RA 17.0 hour and DEC 37.1 deg is visible during the observing window
Object 14670 with RA 23.7 hour and DEC 0.2 deg is visible during the observing window
Object 3871 with RA 3.0 hour and DEC -11.4 deg is visible during the observing window
Object 8095 with RA 15.1 hour and DEC 7.8 deg is visibl